# File Name: simple_agent_building_testing.ipynb
### Location: Chapter 9
### Purpose: 
#####             1. Create the Amazon Bedrock Agent 
#####             2. Attached with Amazon Bedrock Knowledge Base
#####             3. Test the agent
#####             4. Clean up all the resources
##### Dependency: simple_knwl_bases_building.ipynb at Chapter 9 should work properly. 
# <ins>-----------------------------------------------------------------------------------</ins>


# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%%time

%pip install --no-build-isolation --force-reinstall -q \
    "boto3>=1.34.84" \
    "opensearch-py>=2.7.1" \
    "retrying>=1.3.4" \
    "ragas" \
    "ipywidgets>=7.6.5" \
    "iprogress>=0.4" \
    "langchain>=0.2.16" \
    "langchain_community>=0.2.17" \
    "awscli>=1.32.84" \
    "botocore>=1.34.84" \
    "langchain-aws>=0.1.7"    

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import json
import os
import boto3
import botocore
import pprint
import random
from retrying import retry
import warnings
import time
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, RequestError
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
import pprint as pp
from botocore.exceptions import BotoCoreError, ClientError
from datasets import load_dataset
from tqdm import tqdm
import uuid

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

### %store magic command to retrive all the variable value from other notebook. 
### Here, simple_knwl_bases_building.ipynb

In [ ]:
%store -r 

## Define important environment variable

In [ ]:
%%time

# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    boto3_session_name = boto3.session.Session()

    # Create a Bedrock Agent client using the current session and region
    bedrock_agent_client = boto3_session_name.client('bedrock-agent', region_name=aws_region_name)
    
    # Create a Bedrock Agent runtime client using the current session and region
    bedrock_agent_runtime_client = boto3_session_name.client('bedrock-agent-runtime', region_name=aws_region_name)

    # Create an S3 client to interact with Amazon S3
    s3_client = boto3.client('s3')

    # Create an STS client to interact with AWS Security Token Service (STS)
    sts_client = boto3.client('sts')

    # PrettyPrinter instance for formatted output
    pretty_printer = pprint.PrettyPrinter(indent=2)

    # Create an OpenSearch Serverless (AOSS) client using the current session
    aoss_client = boto3_session_name.client('opensearchserverless')
    
    # Create an lambda client
    lambda_client = boto3.client('lambda')


    # Create an IAM client to interact with Identity and Access Management (IAM) service
    iam_client = boto3_session_name.client('iam')

    # Retrieve the current AWS account number and ARN of the caller
    sts_client = boto3.client('sts')
    identity_arn = sts_client.get_caller_identity().get('Arn')
    

    # Embedding model ARN for Bedrock
    embeddingModelArn = f"arn:aws:bedrock:{aws_region_name}::foundation-model/amazon.titan-embed-text-v1"
    
    # Defining information for the Amazon Bedrock agent responsible for video game parlour bookings
    
    bedrock_agent_name = 'video-game-parlour-bookings-agent'  # Name of the Bedrock agent
    bedrock_agent_description = "Agent in charge of managing video game parlour table slot bookings"  # Description of the agent's role
    bedrock_agent_instruction = """
    You are a video game parlour agent responsible for assisting customers with slot booking management. 
    Your tasks include providing information about available slots, creating new bookings, and canceling existing bookings upon request.
    """  # Instructions guiding the agent's tasks and responsibilities

    bedrock_agent_action_group_description = """
    Actions to retrieve information about video game parlour slot bookings, create new bookings, or cancel existing bookings.
    """  # Description of actions that the agent can perform related to bookings

    bedrock_agent_action_group_name = "SlotBookingsActionGroup"  # Name for the action group related to slot bookings

    # Defining the DynamoDB table name. The table has already been created using an AWS CloudFormation template.
    dynamodb_table_name = "video-game-parlour-bookings"  # DynamoDB table for storing booking information

    # Creating a DynamoDB client to interact with Amazon DynamoDB for managing bookings
    dynamodb_client = boto3.client('dynamodb')

    # Defining the Lambda function name. This function has been created using an AWS CloudFormation template.
    lambda_function_name = "video-parlour-bookings-lambda"  # Lambda function for processing bookings

    # Specifying the Amazon Bedrock Foundation Model used for the agent's language processing
    bedrock_llm_foundation_model = "anthropic.claude-3-haiku-20240307-v1:0"  # Model for language understanding and generation

    
    # Store all variables in a dictionary
    variables_store = {
        "aws_region_name": aws_region_name,
        "bedrock_agent_client": bedrock_agent_client,
        "opensearch_service_name": opensearch_service_name,
        "s3_client": s3_client,
        "sts_client": sts_client,
        "aws_account_id": aws_account_id,
        "s3_suffix": s3_suffix,
        "s3_bucket_name": s3_bucket_name,
        "random_suffix": random_suffix,
        "aoss_client": aoss_client,
        "vector_store_name": vector_store_name,
        "index_name": index_name,
        "iam_client": iam_client,
        "sts_client": sts_client,
        "identity_arn": identity_arn,
        "security_policy_name": security_policy_name,
        "network_policy_name": network_policy_name,
        "access_policy_name": access_policy_name,
        "embeddingModelArn": embeddingModelArn,
        "bedrock_knowledge_bases_name": bedrock_knowledge_bases_name,
        "description": description,
        "bedrock_agent_name": bedrock_agent_name,
        "bedrock_agent_description": bedrock_agent_description,
        "bedrock_agent_instruction": bedrock_agent_instruction,
        "bedrock_agent_action_group_description": bedrock_agent_action_group_description,
        "bedrock_agent_action_group_name": bedrock_agent_action_group_name,
        "dynamodb_table_name": dynamodb_table_name,
        "lambda_function_name": lambda_function_name,
        "bedrock_llm_foundation_model": bedrock_llm_foundation_model
    }

    # Print all variables
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")


# Create the Amazon Bedrock Agent

    The code attempts to create a Bedrock agent using specified parameters such as name, role ARN, description, TTL, foundation model, and instructions

In [ ]:
%%time

try:
    # Attempt to create the Bedrock agent
    response = bedrock_agent_client.create_agent(
        agentName=bedrock_agent_name,
        agentResourceRoleArn=genaibookedbedrocksagemakerexecutionrolearn, 
        description=bedrock_agent_description,
        idleSessionTTLInSeconds=1800,
        foundationModel=bedrock_llm_foundation_model,
        instruction=bedrock_agent_instruction,
    )
    
    # Extract agent ID from the response
    agent_id = response['agent']['agentId']
    print("The agent id is:", agent_id)

except bedrock_agent_client.exceptions.ClientError as e:
    # Handle client-side errors (e.g., invalid parameters)
    print(f"An error occurred while creating the agent: {e}")

except KeyError as e:
    # Handle missing keys in the response (e.g., 'agentId' not found)
    print(f"Error extracting agent ID from response: {e}")

except Exception as e:
    # Catch any other unexpected errors
    print(f"An unexpected error occurred: {e}")

# Defining Amazon Bedrock Agent functions. This should be based on Amazon Lambda function lambda_function_name: video-parlour-bookings-lambda

In [ ]:
bedrock_agent_functions = [
    {
        'name': 'get_slot_booking_details',
        'description': 'Retrieve details of a video game parlour slot booking',
        'parameters': {
            "slot_booking_id": {
                "description": "The ID of the slot booking to retrieve",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_slot_booking',
        'description': 'Create a new video game parlour slot booking',
        'parameters': {
            "date": {
                "description": "The date of the slot booking in the format YYYY-MM-DD",
                "required": True,
                "type": "string"
            },
            "name": {
                "description": "Name to person your booking slot",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "The hour of the slot booking in the format  HH:MM",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "The number of guests for the slot booking",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'cancel_slot_booking',
        'description': 'Cancel an existing video game parlour slot booking',
        'parameters': {
            "slot_booking_id": {
                "description": "The slot booking id to cancel",
                "required": True,
                "type": "string"
            }
        }
    },
]

# Find out AWS Lambda function Arn

In [ ]:
%%time

try:
    # Get the Lambda function details using the function name
    response = lambda_client.get_function(FunctionName=lambda_function_name)

    # Extract the ARN from the response
    lambda_function_arn = response['Configuration']['FunctionArn']

    # Print the Lambda function ARN
    print(f"The ARN of the Lambda function is: {lambda_function_arn}")

except lambda_client.exceptions.ResourceNotFoundException:
    print(f"Lambda function '{lambda_function_name}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

# Provide the invoke permission

#### The provided Python code dynamically creates a source ARN for a Bedrock agent and attempts to add a permission to a Lambda function, allowing the Bedrock agent to invoke it.

### Disclaimer: This step can give you error if you are executing twice. You can ignore that. 

In [ ]:
%%time

# Parameters
random_number = random.randrange(200, 900)
bedrock_principal = 'bedrock.amazonaws.com'
bedrock_lambda_statement_id = f"allow-bedrock-agent-invoke-{random_number}"
# Create the source ARN dynamically
source_arn = f'arn:aws:bedrock:{aws_region_name}:{aws_account_id}:agent/{agent_id}'

try:
    # Add permission to invoke Lambda function
    response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        Principal=bedrock_principal,
        StatementId=bedrock_lambda_statement_id,
        Action='lambda:InvokeFunction',
        SourceArn=source_arn
    )

    # Log the successful response
    print(f"Successfully added permission: {response}")
    
except lambda_client.exceptions.ResourceConflictException as e:
    # Handle case where the permission already exists
    print(f"Permission already exists: {e}")
except lambda_client.exceptions.InvalidPermissionValueException as e:
    # Handle invalid permission value error
    print(f"Invalid permission value: {e}")
except Exception as e:
    # Handle any other exception
    print(f"Unexpected error occurred: {e}")

# Create Amazon Bedrock Agent Action Group

### The code attempts to create an action group for a Bedrock agent. It includes a 30-second pause to ensure the agent is provisioned before initiating the action group creation. The create_agent_action_group method uses a Lambda function as an executor, specifies a schema of functions, and includes a descriptive name and description.

In [ ]:
%%time

try:
    # Pause to ensure the agent is created
    time.sleep(30)  # You can adjust this based on your agent's provisioning time

    # Now, create the agent action group
    bedrock_agent_action_group_response = bedrock_agent_client.create_agent_action_group(
        agentId=agent_id,
        agentVersion='DRAFT',  # You can use 'DRAFT' or another version as appropriate
        actionGroupExecutor={
            'lambda': lambda_function_arn
        },
        actionGroupName=bedrock_agent_action_group_name,
        functionSchema={
            'functions': bedrock_agent_functions
        },
        description=bedrock_agent_action_group_description
    )

    # If the action group is created successfully, print the response
    print("Action Group Created Successfully:")
    print(bedrock_agent_action_group_response)

except bedrock_agent_client.exceptions.ClientError as e:
    # Handle client errors (e.g., invalid request, AWS service errors)
    print(f"An error occurred while creating the agent action group: {e}")
    # Optionally, you can log the error for further debugging
except Exception as e:
    # Handle any other unexpected errors
    print(f"An unexpected error occurred: {e}")
    # Optionally, you can log the error for further debugging

# Prepare the Amazon Bedrock agent and check the status to "PREPARED" 

##### This script facilitates the preparation and status monitoring of a Bedrock agent through two key functions. The check_agent_status function retrieves the current status of the agent using bedrock_agent_client.get_agent, and returning the status. The prepare_agent function initiates the preparation process with bedrock_agent_client.prepare_agent and employs a polling mechanism, repeatedly checking the agent's status every 30 seconds until it reaches the PREPARED state, while providing progress updates.

In [ ]:
%%time

# Function to check the agent status
def check_agent_status(agent_id):
    try:
        # Attempt to retrieve the agent's status
        response = bedrock_agent_client.get_agent(agentId=agent_id)
        
        # Extract the current status from the response
        agent_status = response['agent']['agentStatus']
        print(f"Current agent status: {agent_status}")
        
        return agent_status
    except ClientError as e:
        print(f"Error retrieving agent status: {e}")
        return None

# Function to prepare the agent
def prepare_agent(agent_id):
    try:
        # Prepare the agent
        response = bedrock_agent_client.prepare_agent(agentId=agent_id)
        print("Agent preparation initiated:", response)
        
        # Pause for 30 seconds to ensure the agent is being prepared
        time.sleep(30)

        # Loop until the agent status is 'PREPARED'
        while True:
            agent_status = check_agent_status(agent_id)
            
            # If the agent status is 'PREPARED', break out of the loop
            if agent_status == 'PREPARED':
                print("Agent is prepared and ready!")
                break
            else:
                # Pause for 30 seconds before checking the status again
                print("Agent is not prepared yet. Waiting...")
                time.sleep(30)
    
    except ClientError as e:
        print(f"Error preparing agent: {e}")

# Call the prepare_agent function
prepare_agent(agent_id)

# This code snippet demonstrates associating a Bedrock agent with a knowledge base

In [ ]:
%%time

try:
        # Making the API call to associate the agent with the knowledge base
        response = bedrock_agent_client.associate_agent_knowledge_base(
            agentId=agent_id,
            agentVersion='DRAFT',
            description='Access the knowledge base when customers ask about the plates in the menu.',
            knowledgeBaseId=knowledgeBaseId,  # Specify the knowledge base ID
            knowledgeBaseState='ENABLED'    # Set the knowledge base state as ENABLED
        )

        # Log the successful response
        print(f"Knowledge base successfully associated with agent. Response: {response}")

except bedrock_agent_client.exceptions.ResourceNotFoundException as e:
    # Handle the case when the resource is not found (e.g., agent or knowledge base)
    logger.error(f"Resource not found: {e}")
    raise

except bedrock_agent_client.exceptions.InvalidInputException as e:
    # Handle the case when the input parameters are invalid
    logger.error(f"Invalid input: {e}")
    raise

except Exception as e:
    # Catch any other exceptions and log them
    logger.error(f"Unexpected error occurred while associating agent with knowledge base: {str(e)}")
    raise


## The get_agent_responses function is designed to invoke a Bedrock agent with a query and session information, handle its responses, and manage session states.

In [ ]:
%%time

def get_agent_responses(query, session_id, agent_id, alias_id, enable_trace=False, session_state=None):
    """
    Function to invoke an agent with the provided query and session information.

    Parameters:
        query (str): The input text/query to send to the agent.
        session_id (str): The unique identifier for the session.
        agent_id (str): The unique identifier for the agent.
        alias_id (str): The alias ID for the agent.
        session_state (dict): Optional session state to maintain between requests.

    """
    # Default session_state to an empty dictionary if not provided
    if not session_state:
        session_state = {}

    # Flag to indicate whether the session should end
    end_session = False

    try:
        # Invoke the agent API
        agent_response = bedrock_agent_runtime_client.invoke_agent(
            inputText=query,
            agentId=agent_id,
            agentAliasId=alias_id,
            sessionId=session_id,
            enableTrace=enable_trace,
            endSession=end_session,
            sessionState=session_state
        )
        
        completion = ""

        for event in agent_response.get("completion"):
            chunk = event["chunk"]
            completion = completion + chunk["bytes"].decode()
            
        print(completion)

    except ValueError as ve:
        # Log specific error for unexpected event type and raise it
        print(f"Error occurred: {ve}")
        raise

    except Exception as e:
        # Log any other general exception and raise it
        print(f"Unexpected error occurred: {str(e)}")
        raise Exception("Unexpected error while invoking agent.", e)


# Create alias for Amazon Bedrock Agent 

In [ ]:
%%time

try:
    # Define alias name and description
    bedrock_alias_name = 'video-game-parlour-bookings-alias'
    bedrock_alias_description = 'video game parlour bookings agent alias'

    # Attempt to create the agent alias
    response = bedrock_agent_client.create_agent_alias(
        agentAliasName=bedrock_alias_name,
        agentId=agent_id,
        description=bedrock_alias_description
    )

    # Print the successful response
    print(f"Agent alias created successfully: {response}")
    
    alias_id = response["agentAlias"]["agentAliasId"]

except bedrock_agent_client.exceptions.ResourceNotFoundException as e:
    # Handle the case where the specified agent does not exist
    print(f"Error: Agent not found for the given agent ID: {e}")

except bedrock_agent_client.exceptions.InvalidInputException as e:
    # Handle invalid input errors (e.g., malformed alias name or missing fields)
    print(f"Error: Invalid input provided while creating alias: {e}")

except bedrock_agent_client.exceptions.ResourceConflictException as e:
    # Handle the case where an alias with the same name already exists
    print(f"Error: Alias with the name '{bedrock_alias_name}' already exists: {e}")

except Exception as e:
    # Catch and handle any other unexpected errors
    print(f"An unexpected error occurred while creating the agent alias: {str(e)}")
    raise

# Test the Amazon Bedrock Agent

#### Use case 1: Retriving information about Video Game parlour from knowledge Base information with in a session and agent identity

In [ ]:
session_id:str = str(uuid.uuid1())
agent_identity = "Mr X"

In [ ]:
%%time 

try:
    # Define the query and session state
    query = "Suggest a game with Solve puzzles from a virtual reality prison."
    session_state = {
        "promptSessionAttributes": {
            "name": agent_identity
        }
    }

    # Invoke the function to get agent responses
    response = get_agent_responses(query, session_id, agent_id, alias_id, session_state=session_state)
    print("\n\n")

except ValueError as ve:
    # Handle ValueError if something is wrong with the input or response data
    print(f"ValueError occurred: {ve}")

except KeyError as ke:
    # Handle KeyError if required keys are missing in the response
    print(f"KeyError occurred: {ke}")

except bedrock_agent_runtime_client.exceptions.ClientError as ce:
    # Handle client-specific errors (e.g., invalid agent, alias, or session IDs)
    print(f"ClientError occurred: {ce}")

except Exception as e:
    # Catch and handle any other unexpected errors
    print(f"An unexpected error occurred: {str(e)}")
    raise

#### Use case 2: Video Game parlour slot booking with in a session and agent identity

In [ ]:
%%time 

try:
    # Define the query and session state
    query = "I want to create a slot booking for 2 guests, at 8pm on the 6th of May 2024."
    session_state = {
        "promptSessionAttributes": {
            "name": agent_identity
        }
    }

    # Invoke the function to get the agent's response
    response = get_agent_responses(query, session_id, agent_id, alias_id, session_state=session_state)
    print("\n\n")

except ValueError as ve:
    # Handle ValueError for input/output issues
    print(f"ValueError occurred: {ve}")

except KeyError as ke:
    # Handle KeyError for missing keys in the response
    print(f"KeyError occurred: {ke}")

except bedrock_agent_runtime_client.exceptions.ClientError as ce:
    # Handle client-specific errors (e.g., invalid session, alias, or agent)
    print(f"ClientError occurred: {ce}")

except Exception as e:
    # Handle unexpected errors
    print(f"An unexpected error occurred: {str(e)}")
    raise


#### Use case 3: Video Game parlour slot booking enquiry in a session and agent identity

In [ ]:
## provide this value from the result of above execution 
slot_booking_id = "<PROVIDE THE SLOT BOOKING ID HERE FROM ABOVE>"

In [ ]:
%%time

try:
    # Define the query to check for a slot booking
    query = f"Hi, Do I have any slot booking with Id {slot_booking_id}?"
    
    # Define the session state
    session_state = {
        "promptSessionAttributes": {
            "name": agent_identity  # Make sure agent_identity is defined earlier
        }
    }
    
    # Call the function to get the agent's response
    response = get_agent_responses(query, session_id, agent_id, alias_id, session_state=session_state)
    print("\n\n")

except ValueError as ve:
    # Handle ValueError for invalid response data
    print(f"ValueError occurred: {ve}")

except KeyError as ke:
    # Handle KeyError for missing keys in the response
    print(f"KeyError occurred: {ke}")

except bedrock_agent_runtime_client.exceptions.ClientError as ce:
    # Handle client-specific errors (e.g., issues with the agent, session, or alias)
    print(f"ClientError occurred: {ce}")

except Exception as e:
    # Catch all other unexpected exceptions
    print(f"An unexpected error occurred: {str(e)}")
    raise

#### Use case 4: Video Game parlour slot booking cancel in a session and agent identity

In [ ]:
%%time

try:
    
    # Dynamically format the query to cancel the slot booking
    query = f"Hi, I want to cancel the slot booking with Id {slot_booking_id}."
    
    # Define the session state
    session_state = {
        "promptSessionAttributes": {
            "name": agent_identity  # Make sure agent_identity is defined earlier
        }
    }
    
    # Call the function to get the agent's response
    response = get_agent_responses(query, session_id, agent_id, alias_id, session_state=session_state)
    print("\n\n")

except ValueError as ve:
    # Handle ValueError for invalid response data
    print(f"ValueError occurred: {ve}")

except KeyError as ke:
    # Handle KeyError for missing keys in the response
    print(f"KeyError occurred: {ke}")

except bedrock_agent_runtime_client.exceptions.ClientError as ce:
    # Handle client-specific errors (e.g., issues with the agent, session, or alias)
    print(f"ClientError occurred: {ce}")

except Exception as e:
    # Catch all other unexpected exceptions
    print(f"An unexpected error occurred: {str(e)}")
    raise

# End of NoteBook 

#### <ins>Step 1</ins> 

##### Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

##### Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html


#### <ins>Step 2</ins> 

#### If you are not executing any further lab of this Chapter 9
##### uncommet and execute all the below code. 

# Delete Amazon Bedrock Knowledge Bases and corresponding data sources

##### This code effectively manages the process of listing and deleting all data sources associated with a specified KnowledgeBase in Bedrock.

In [ ]:
'''%%time 

def list_data_sources(client, knowledge_base_id):
    """
    Fetch the data sources associated with the given KnowledgeBase.
    
    Parameters:
    - client: Bedrock client instance
    - knowledge_base_id: The ID of the KnowledgeBase
    
    Returns:
    - A list of data sources summaries or None if an error occurs.
    """
    try:
        response = client.list_data_sources(knowledgeBaseId=knowledge_base_id)
        data_sources_list = response.get('dataSourceSummaries', [])
        print(f"Successfully retrieved {len(data_sources_list)} data source(s).")
        return data_sources_list
    except Exception as e:
        print(f"Error listing data sources: {e}")
        return None

def delete_data_source(client, data_source_id, knowledge_base_id):
    """
    Delete a specific data source from the KnowledgeBase.
    
    Parameters:
    - client: Bedrock client instance
    - data_source_id: The ID of the data source to delete
    - knowledge_base_id: The ID of the KnowledgeBase
    
    Returns:
    - True if deletion was successful, False otherwise.
    """
    try:
        client.delete_data_source(dataSourceId=data_source_id, knowledgeBaseId=knowledge_base_id)
        print(f"Successfully deleted data source: {data_source_id}")
        return True
    except Exception as e:
        print(f"Error deleting data source {data_source_id}: {e}")
        return False

def delete_all_data_sources(client, knowledge_base_id):
    """
    Delete all data sources associated with the specified KnowledgeBase.
    
    Parameters:
    - client: Bedrock client instance
    - knowledge_base_id: The ID of the KnowledgeBase
    """
    # Step 1: List all data sources
    data_sources_list = list_data_sources(client, knowledge_base_id)
    if data_sources_list is None:
        return  # Exit if listing data sources failed

    # Step 2: Delete each data source
    for idx, ds in enumerate(data_sources_list):
        data_source_id = ds.get("dataSourceId")
        kb_id = ds.get("knowledgeBaseId")

        # Deleting the data source
        if delete_data_source(client, data_source_id, kb_id):
            # Wait for 10 seconds between each deletion to avoid rate limiting issues
            time.sleep(10)

# Assuming `bedrock_agent_client` and `knowledgeBaseId` are initialized
delete_all_data_sources(bedrock_agent_client, knowledgeBaseId)'''

# Delete Amazon OpenSearch serverless Collection using its ARN 

In [ ]:
'''%%time 

def extract_collection_id_from_arn(arn):
    """Extracts the collection ID from the OpenSearch Serverless ARN."""
    try:
        # The collection ID is the last segment after the "/" in the ARN
        collection_id = arn.split("/")[-1]
        return collection_id
    except Exception as e:
        print(f"Error extracting collection ID from ARN: {e}")
        return None

def delete_opensearch_collection(aoss_client, collection_arn):
    """Deletes an OpenSearch Serverless collection using its ARN."""
    # Step 1: Extract the collection ID from the ARN
    collection_id = extract_collection_id_from_arn(collection_arn)
    
    if collection_id is None:
        print("Invalid collection ARN. Cannot proceed with deletion.")
        return
    
    # Step 2: Use the OpenSearch client to delete the collection by ID
    try:
        response = aoss_client.delete_collection(id=collection_id)
        print(f"Collection {collection_id} deleted successfully.")
    except Exception as e:
        print(f"Error deleting collection: {e}")

# Call the function to delete the collection
delete_opensearch_collection(aoss_client, aoss_collectionarn)'''

In [ ]:
'''%%time

# Finally delete the knowledge base
try:
    # Attempt to delete the specified knowledge base
    response = bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=knowledgeBaseId)
    
    # Log the successful response
    print(f"Knowledge Base deleted successfully: {response}")

except bedrock_agent_client.exceptions.ResourceNotFoundException as e:
    # Handle the case where the Knowledge Base does not exist
    print(f"Knowledge Base with ID {knowledgeBaseId} not found: {e}")

except bedrock_agent_client.exceptions.AccessDeniedException as e:
    # Handle insufficient permissions
    print(f"Access denied when attempting to delete Knowledge Base: {e}")

except bedrock_agent_client.exceptions.ValidationException as e:
    # Handle invalid input or parameters
    print(f"Invalid input for Knowledge Base deletion: {e}")

except Exception as e:
    # Catch any other unexpected errors
    print(f"An unexpected error occurred: {e}")'''

In [ ]:
'''%%time

try:
    # Delete the agent alias first
    response = bedrock_agent_client.delete_agent_alias(
        agentAliasId=alias_id,  # Alias ID of the agent to delete
        agentId=agent_id  # Agent ID to delete
    )
    print(f"Agent alias {alias_id} deleted successfully.")

    # Now delete the agent itself
    response = bedrock_agent_client.delete_agent(
        agentId=agent_id  # The actual agent ID to delete
    )
    
    # Log the successful response
    print(f"Agent ID {agent_id} deleted successfully: {response}")

except bedrock_agent_client.exceptions.ResourceNotFoundException as e:
    # Handle the case where the resource (agent or alias) is not found
    print(f"Agent with ID {agent_id} or alias with ID {alias_id} not found: {e}")

except bedrock_agent_client.exceptions.AccessDeniedException as e:
    # Handle insufficient permissions error
    print(f"Access denied when attempting to delete Agent ID {agent_id}: {e}")

except bedrock_agent_client.exceptions.ValidationException as e:
    # Handle invalid input errors for agent deletion
    print(f"Invalid input for Agent ID {agent_id} deletion: {e}")

except Exception as e:
    # Catch any other unexpected errors
    print(f"An unexpected error occurred: {e}")'''